# **Import libraries**

In [1]:
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Audio
from sklearn.preprocessing import StandardScaler
from PIL import Image,ImageOps
from io import BytesIO
import shutil
import seaborn as sns
import librosa
import shutil
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.optimizers import Adam

2024-02-26 22:59:37.534114: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-26 22:59:37.534239: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-26 22:59:37.663440: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# **letters**

In [ ]:
def SpectrogramGenerator(signal,sr,label,audio_name):
    trimmed_signal, _ = librosa.effects.trim(signal)
    normalized_audio = librosa.util.normalize(trimmed_signal)
    duration=int(1*sr)
    padded_audio = librosa.util.fix_length(trimmed_signal, size=duration)
    mfcc_feature = librosa.feature.mfcc(y=signal, sr=16000, n_fft=1024,hop_length=30,n_mfcc=13)
    scaler = StandardScaler()
    mfcc_feature_standardized = scaler.fit_transform(mfcc_feature)
    plt.figure(figsize=(1, 1))
    librosa.display.specshow(librosa.amplitude_to_db(mfcc_feature_standardized,ref=np.max))
    plt.colorbar(format='%+2.0f dB').remove()
    buf = BytesIO()
    plt.savefig(buf, format='png', bbox_inches='tight', pad_inches=0, transparent=True, dpi=100)
    buf.seek(0)
    image_name = f"{audio_name}_spectrogram.png"
    image_path = os.path.join(os.path.join("/kaggle/working/letters_spectrograms/",label), image_name)
    img = Image.open(buf)
    img = img.resize((200, 200), Image.LANCZOS)
    img.save(image_path, format='png')
    plt.close()
    buf.close() 

In [ ]:
os.makedirs("/kaggle/working/letters_spectrograms", exist_ok=True)
data_path = '/kaggle/input/new-audio-arabic-letters/Dataset يارب نخلص'
labels = os.listdir(data_path)
for label in labels:
    label_path = os.path.join(data_path, label)
    os.makedirs(os.path.join("/kaggle/working/letters_spectrograms",label), exist_ok=True)
    for audio_name in os.listdir(label_path):
        audio_path = os.path.join(label_path, audio_name)
        signal,sr=librosa.load(audio_path,sr=16000)
        y_stretch = librosa.effects.time_stretch(signal, rate=1.2)
        n_steps = 4
        y_pitch_shift = librosa.effects.pitch_shift(signal, n_steps=n_steps, sr=sr)
        noise = np.random.randn(len(signal))
        y_noise = signal + 0.005 * noise
        y_speed_up = librosa.effects.time_stretch(signal, rate=1.5)
        y_slow_down = librosa.effects.time_stretch(signal, rate=0.75)
        y_reverb = librosa.effects.preemphasis(signal)
        SpectrogramGenerator(signal,sr,label,audio_name)
        SpectrogramGenerator(y_stretch,sr,label,audio_name+'_stretch')
        SpectrogramGenerator(y_pitch_shift,sr,label,audio_name+'_pitch_shift')
        SpectrogramGenerator(y_noise,sr,label,audio_name+'_noise')
        SpectrogramGenerator(y_speed_up,sr,label,audio_name+'_speed_up')
        SpectrogramGenerator(y_slow_down,sr,label,audio_name+'_slow_down')
        SpectrogramGenerator(y_reverb,sr,label,audio_name+'_reverb')
        plt.close()
for label in labels:
    label_path = os.path.join(data_path, label)
    for audio_name in os.listdir(label_path):
        audio_path = os.path.join(label_path, audio_name)
        signal,sr=librosa.load(audio_path,sr=16000)
        y_stretch = librosa.effects.time_stretch(signal, rate=.9)
        n_steps = 8
        y_pitch_shift = librosa.effects.pitch_shift(signal, n_steps=n_steps, sr=sr)
        noise = np.random.randn(len(signal))
        y_noise = signal + 0.01 * noise
        y_speed_up = librosa.effects.time_stretch(signal, rate=2)
        y_slow_down = librosa.effects.time_stretch(signal, rate=0.5)
        y_reverb = librosa.effects.preemphasis(signal)
        SpectrogramGenerator(signal,sr,label,audio_name)
        SpectrogramGenerator(y_stretch,sr,label,audio_name+'_stretch1')
        SpectrogramGenerator(y_pitch_shift,sr,label,audio_name+'_pitch_shift1')
        SpectrogramGenerator(y_noise,sr,label,audio_name+'_noise1')
        SpectrogramGenerator(y_speed_up,sr,label,audio_name+'_speed_up1')
        SpectrogramGenerator(y_slow_down,sr,label,audio_name+'_slow_down1')
        SpectrogramGenerator(y_reverb,sr,label,audio_name+'_reverb1')
        plt.close()

In [ ]:
shutil.make_archive('/kaggle/working/letters_spectrograms', 'zip', '/kaggle/working')

# **Vowels**

In [ ]:
os.makedirs("/kaggle/working/vowels_spectrograms", exist_ok=True)
data_path = '/kaggle/input/arabic-short-vowels-audio-dataset/alphabet_new_dataset'
labels = os.listdir(data_path)
for label in labels:
    label_path = os.path.join(data_path, label)
    os.makedirs(os.path.join("/kaggle/working/vowels_spectrograms",label), exist_ok=True)
    for audio_name in os.listdir(label_path):
        audio_path = os.path.join(label_path, audio_name)
        signal,sr=librosa.load(audio_path,sr=16000)
        trimmed_signal, _ = librosa.effects.trim(signal)
        normalized_audio = librosa.util.normalize(trimmed_signal)
        duration=int(1*sr)
        padded_audio = librosa.util.fix_length(trimmed_signal, size=duration)
        mfcc_feature = librosa.feature.mfcc(y=signal, sr=16000, n_mfcc=40)
        scaler = StandardScaler()
        mfcc_feature_standardized = scaler.fit_transform(mfcc_feature)
        plt.figure(figsize=(1, 1))
        librosa.display.specshow(librosa.amplitude_to_db(mfcc_feature_standardized,ref=np.max))
        plt.colorbar(format='%+2.0f dB').remove()
        buf = BytesIO()
        plt.savefig(buf, format='png', bbox_inches='tight', pad_inches=0, transparent=True, dpi=100)
        buf.seek(0)
        image_name = f"{audio_name}_spectrogram.png"
        image_path = os.path.join(os.path.join("/kaggle/working/vowels_spectrograms/",label), image_name)
        img = Image.open(buf)
        img = img.resize((150, 150), Image.LANCZOS)
        img.save(image_path, format='png')
        plt.close()
        buf.close() 
shutil.make_archive('/kaggle/working/vowels_spectrograms', 'zip', '/kaggle/working')